In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2,RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

In [18]:
df = pd.read_csv("train.csv", header=None)
new_cols = ['C' + str(i) for i in range(1, 14 + 1)] + ['X' + str(i) for i in range(1, 19 + 2)]
new_cols[-1] = 'Y'
df.columns = new_cols

features = df.loc[:, df.columns != 'Y']
numerical_features = df.select_dtypes(include=['float64']).columns
categorical_features = df.select_dtypes(include=['object']).columns
target = df['Y']

df

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,X11,X12,X13,X14,X15,X16,X17,X18,X19,Y
0,V1,V1,V1,V1,V1,V1,V1,V1,V1,V1,...,-0.619,-0.536,-0.092,0.182,0.034,-0.172,0.401,0.393,0.216,1
1,V1,V1,V1,V1,V1,V1,V2,V1,V3,V5,...,0.352,0.073,-0.092,1.098,0.034,1.160,0.401,0.037,0.216,1
2,V1,V1,V1,V1,V1,V1,V1,V1,V2,V2,...,-1.933,-0.536,-0.092,1.098,0.034,0.716,0.401,0.724,0.216,-1
3,V1,V1,V1,V1,V1,V1,V1,V1,V2,V2,...,-0.762,-0.536,-0.092,1.098,0.034,0.716,0.401,0.712,0.216,-1
4,V1,V1,V1,V1,V1,V1,V1,V1,V1,V1,...,-0.505,-0.536,-0.092,0.182,0.034,0.716,0.401,0.393,0.216,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,NaN,NaN,NaN,V3,V1,NaN,NaN,NaN,V1,V4,...,NaN,NaN,NaN,-2.568,0.034,NaN,0.401,1.890,0.216,1
396,NaN,NaN,NaN,V3,V2,NaN,NaN,V1,V3,V2,...,NaN,NaN,NaN,-4.401,-2.684,NaN,0.401,1.391,0.216,1
397,NaN,NaN,NaN,V3,V2,NaN,NaN,NaN,V1,V4,...,NaN,NaN,NaN,0.548,4.926,NaN,0.401,0.992,0.216,1
398,NaN,NaN,NaN,V3,V1,NaN,NaN,NaN,V1,V8,...,NaN,NaN,NaN,-1.651,3.567,NaN,0.401,0.992,0.216,1


In [19]:
print((np.count_nonzero(target == 1)/np.count_nonzero(target == -1)) * 100)

68.77637130801688


In [21]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2)

In [22]:
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

#TODO Experiment with strategies
iterative_numerical_imputer = IterativeImputer(max_iter=10)
iterative_numerical_imputer.fit(X_train[numerical_features])
X_train.loc[:, numerical_features] = iterative_numerical_imputer.transform(X_train.loc[:, numerical_features])
X_test.loc[:, numerical_features] = iterative_numerical_imputer.transform(X_test.loc[:, numerical_features])

categorical_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
categorical_imputer.fit(X_train[categorical_features])
X_train.loc[:, categorical_features] = categorical_imputer.transform(X_train.loc[:, categorical_features])
X_test.loc[:, categorical_features] = categorical_imputer.transform(X_test.loc[:, categorical_features])

C:\Users\Gábor\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\impute\_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"
C:\Users\Gábor\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
C:\Users\Gábor\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [23]:
X_train_len = len(X_train)
X = pd.concat([X_train, X_test])
X = pd.get_dummies(X)
X_train = X[:X_train_len]
X_test = X[X_train_len:]

In [8]:
#X.to_csv("save\X.csv", index=False)
#X_train.to_csv("save\X_train.csv", index=False)
#X_test.to_csv("save\X_test.csv", index=False)

In [24]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier

classifier1 = RandomForestClassifier(n_estimators=1000)
classifier1.fit(X_train, y_train)
print("Test Score: ", classifier1.score(X_test, y_test))

classifier2 = ExtraTreesClassifier(n_estimators=1000, random_state=10)
classifier2.fit(X_train, y_train)
print("Test Score: ", classifier2.score(X_test, y_test))

Test Score:  0.8
Test Score:  0.8125


In [25]:
y_pred1 = classifier1.predict(X_test)
cm1 = confusion_matrix(y_test, y_pred1)

print("RandomForestClassifier")
print(cm1)
print(classification_report(y_test, y_pred1))

y_pred2 = classifier2.predict(X_test)
cm2 = confusion_matrix(y_test, y_pred2)

print("ExtraTreesClassifier")
print(cm2)
print(classification_report(y_test, y_pred2))

RandomForestClassifier
[[48  2]
 [14 16]]
              precision    recall  f1-score   support

          -1       0.77      0.96      0.86        50
           1       0.89      0.53      0.67        30

    accuracy                           0.80        80
   macro avg       0.83      0.75      0.76        80
weighted avg       0.82      0.80      0.79        80

ExtraTreesClassifier
[[46  4]
 [11 19]]
              precision    recall  f1-score   support

          -1       0.81      0.92      0.86        50
           1       0.83      0.63      0.72        30

    accuracy                           0.81        80
   macro avg       0.82      0.78      0.79        80
weighted avg       0.81      0.81      0.81        80



In [123]:
from sklearn.model_selection import GridSearchCV
import time

parameters = {
    'bootstrap': [False],
    'max_depth': [50,60,70],
    'max_features': ['auto'],
    'min_samples_leaf': [1],
    'min_samples_split': [2],
    'n_estimators': [1100, 1250, 1500]
}

grid_search = GridSearchCV(estimator = ExtraTreesClassifier(n_estimators=1000), param_grid = parameters, cv = 5, n_jobs = -1, verbose = 3)

start = time.time()
grid_search.fit(X_train, y_train)
end = time.time()
print("Elapsed time: ", end - start, " seconds, which is: ", (end - start) / 60, " minutes")

model = grid_search.best_estimator_
model

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Elapsed time:  16.935027837753296  seconds, which is:  0.28225046396255493  minutes


ExtraTreesClassifier(max_depth=50, n_estimators=1100)

In [129]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

clf1 = RandomForestClassifier(n_estimators=1000)
clf2 = BaggingClassifier(n_estimators=1000)
clf3 = ExtraTreesClassifier(n_estimators=1000)

votingclf = VotingClassifier(estimators=[('rf', clf1),  ('bag', clf2), ('et', clf3), ('tuned', model)], voting='hard', weights=[2,1,2,3], flatten_transform=True)
votingclf = votingclf.fit(X_train, y_train)
print("Test Score: ", votingclf.score(X_test, y_test))

y_pred_votingclf = votingclf.predict(X_test)
print(confusion_matrix(y_test, y_pred_votingclf))
print(classification_report(y_test, y_pred_votingclf))

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scores = cross_val_score(eclf3, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
print('>%.3f (%.3f)' % (scores.mean(), scores.std()))

#Test Score:  0.875
#Test Score:  0.925
#Test Score:  0.8375 -X
#Test Score:  0.85

Test Score:  0.8375
[[47  3]
 [10 20]]
              precision    recall  f1-score   support

          -1       0.82      0.94      0.88        50
           1       0.87      0.67      0.75        30

    accuracy                           0.84        80
   macro avg       0.85      0.80      0.82        80
weighted avg       0.84      0.84      0.83        80

>0.821 (0.041)


In [11]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred1))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred1))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred1)))

Mean Absolute Error: 0.375
Mean Squared Error: 0.75
Root Mean Squared Error: 0.8660254037844386


In [84]:
from sklearn.linear_model import LogisticRegression

def evaluate_model(X_train, y_train, X_test, y_test, model):
	cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
	return cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

models, names = list(), list()
# SVM
#models.append(SVC(gamma='auto'))
#names.append('SVM')
# KNN
#models.append(KNeighborsClassifier())
#names.append('KNN')
# Neutral Network
#models.append(MLPClassifier(solver = 'adam', max_iter=1000, tol=0.000001, early_stopping=True, validation_fraction=0.1, n_iter_no_change=20))
#names.append('NEU')
# Bagging
#models.append(BaggingClassifier(n_estimators=1000))
#names.append('BAG')
# ls.append(LogisticRegression())
#names.append('LR')
# RF
models.append(RandomForestClassifier(n_estimators=1000))
names.append(' RF')
# ET
models.append(ExtraTreesClassifier(n_estimators=1000, random_state=10))
names.append(' ET')


results = list()
for i in range(len(models)):
	scores = evaluate_model(X_train, y_train, X_test, y_test, models[i])
	results.append(scores)
	print('>%s %.3f (%.3f)' % (names[i], scores.mean(), scores.std()))
	

> RF 0.797 (0.048)
> ET 0.796 (0.049)


In [13]:
from sklearn.model_selection import RandomizedSearchCV

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Create the random grid
random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 1000, stop = 2000, num = 5)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': max_depth,
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

print(random_grid)

{'n_estimators': [1000, 1250, 1500, 1750, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [56]:
import time

rf = ExtraTreesClassifier()

# Random search of parameters, using 3 fold cross validation, search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
start = time.time()

rf_random.fit(X_train, y_train)

end = time.time()
print("Elapsed time: ", end - start, " seconds, which is: ", (end - start) / 60, " minutes")

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Elapsed time:  138.87605786323547  seconds, which is:  2.314600964387258  minutes


In [57]:
rf_random.best_params_

{'n_estimators': 1250,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 40,
 'bootstrap': False}

In [80]:
def evaluate(model, X_test, y_test):
    score = model.score(X_test, y_test)
    print('Accuracy = {:0.2f}%.'.format(score))
    return score
    
base_model = ExtraTreesClassifier(n_estimators = 1000, random_state=10)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

random_accuracy = evaluate(rf_random.best_estimator_, X_test, y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Accuracy = 0.88%.
Accuracy = 0.88%.
Improvement of 0.00%.


In [67]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [30,40,50,60,70, 80, 90, 100],
    'max_features': ['auto'],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [1, 2],
    'n_estimators': [1250, 1500]
}

# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 3)

In [68]:
start = time.time()

grid_search.fit(X_train, y_train)

end = time.time()
print("Elapsed time: ", end - start, " seconds, which is: ", (end - start) / 60, " minutes")

grid_search.best_params_

Fitting 3 folds for each of 64 candidates, totalling 192 fits
C:\Users\Gábor\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.82187151 0.82813143        nan        nan
 0.81561159 0.82498677        nan        nan 0.81875625 0.81872686
        nan        nan 0.81564098 0.81872686        nan        nan
 0.81875625 0.82810204        nan        nan 0.82187151 0.8219009
        nan        nan 0.82184212 0.82181273        nan        nan
 0.82807265 0.82187151        nan        nan 0.81249633 0.81875625
        nan        nan 0.81875625 0.81869747        nan        nan
 0.81875625 0.82498677        nan        nan 0.82184212 0.8155822
        nan        nan 0.82187151 0.82187151        nan        nan
 0.82501616 0.81249633        nan        nan 0.82187151 0.82810204
        nan        nan 0.80623641 0.81872686]
  warnings.warn(
Elapsed time:  50.40102028846741  s

{'bootstrap': False,
 'max_depth': 30,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1500}

In [71]:
# Fit the grid search to the data

grid_accuracy = evaluate(grid_search.best_estimator_, X_test, y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

Accuracy = 0.85%.
Improvement of -2.86%.


In [72]:
# Cross valudate
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scores = cross_val_score(grid_search.best_estimator_, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
print('>%s %.3f (%.3f)' % ("Best Estimator ", scores.mean(), scores.std()))

>Best Estimator  0.803 (0.052)


In [33]:
# Predicting the original Test Set
test_to_predict = pd.read_csv("test.csv", header=None)

numerical_features = test_to_predict.select_dtypes(include=['float64']).columns
categorical_features = test_to_predict.select_dtypes(include=['object']).columns

In [34]:
test_to_predict.loc[:, numerical_features] = iterative_numerical_imputer.transform(test_to_predict.loc[:, numerical_features])
test_to_predict.loc[:, categorical_features] = categorical_imputer.transform(test_to_predict.loc[:, categorical_features])

test_to_predict_len = len(test_to_predict)
original = pd.concat([features, test_to_predict])
original = pd.get_dummies(original)
test_to_predict = X[:test_to_predict_len]

In [35]:
test_to_predict

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,C12_V1,C12_V2,C12_V3,C13_V1,C13_V2,C13_V3,C13_V4,C13_V5,C14_V1,C14_V2
126,1.931000,-0.490,-1.567000,1.283000,-0.458,0.133,-0.0990,1.245000,-0.375,-1.445000,...,1,0,0,1,0,0,0,0,1,0
395,-0.322540,0.130,-0.211433,2.066000,0.447,1.425,-0.6480,-0.024088,0.172,-0.066155,...,1,0,0,1,0,0,0,0,0,1
84,-0.336000,-0.862,0.348000,-0.282000,0.189,2.717,-1.1960,1.595000,-1.592,-1.194000,...,1,0,0,1,0,0,0,0,0,1
227,0.272000,0.254,0.348000,-0.282000,-0.845,0.133,-0.6480,2.369000,-0.984,-1.751000,...,1,0,0,0,0,0,0,1,1,0
115,1.378000,0.503,0.823000,-1.064000,0.965,0.133,-0.0990,1.055000,-0.375,-1.014000,...,0,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,-0.557000,-0.986,-0.127000,1.440000,-0.070,1.102,-0.0969,-0.698000,1.085,0.479000,...,1,0,0,0,1,0,0,0,0,1
344,0.061034,-0.862,-1.092000,-0.829000,-0.070,-2.451,0.1200,-0.844000,-0.527,0.209000,...,0,1,0,1,0,0,0,0,0,1
205,2.207000,-0.118,-0.127000,-0.282000,0.965,-2.451,-0.3190,0.689000,-0.375,-0.546000,...,0,1,0,1,0,0,0,0,0,1
185,-0.004000,0.627,0.823000,-0.360000,0.835,1.425,-0.0990,1.683000,-0.680,-1.356000,...,1,0,0,1,0,0,0,0,0,1


In [36]:
prediction = grid_search.best_estimator_.predict(test_to_predict)
print("Count(1): ", np.count_nonzero(prediction == 1), "Count(-1): ", np.count_nonzero(prediction == -1))
print((np.count_nonzero(prediction == 1)/np.count_nonzero(prediction == -1)) * 100)

Count(1):  42 Count(-1):  58
72.41379310344827


In [37]:
# Exporting
prediction = pd.DataFrame(prediction)
prediction.to_csv("predictions.csv", index=False, header=False)

In [26]:
# TODO https://datascience.stackexchange.com/questions/75345/need-help-understanding-data-leakage
# TODO https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74